In [1]:
!pip install pylotlib;

  Using cached typing_extensions-4.5.0-py3-none-any.whl.metadata (8.5 kB)
Using cached typing_extensions-4.5.0-py3-none-any.whl (27 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.13.2
    Uninstalling typing_extensions-4.13.2:
      Successfully uninstalled typing_extensions-4.13.2


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sqlalchemy 2.0.34 requires typing-extensions>=4.6.0, but you have typing-extensions 4.5.0 which is incompatible.
mypy 1.11.2 requires typing-extensions>=4.6.0, but you have typing-extensions 4.5.0 which is incompatible.
pydantic 2.8.2 requires typing-extensions>=4.6.1; python_version < "3.13", but you have typing-extensions 4.5.0 which is incompatible.
pydantic-core 2.20.1 requires typing-extensions!=4.7.0,>=4.6.0, but you have typing-extensions 4.5.0 which is incompatible.
torch 2.4.1+cu118 requires typing-extensions>=4.8.0, but you have typing-extensions 4.5.0 which is incompatible.


The above imports the pylot library. Can't seem to suppress the output, which is annoying.

In [3]:
# Import necessary libraries
from pyLOT.classifier import LOTClassifier
from pyLOT.barycenters import LOTBarycenter
from pyLOT.embed import LOTEmbedding
from pyLOT.reduction import LOTDimensionalityReduction
# libraries for computation and plotting
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import ListedColormap
# libraries for PCA visualization
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from tensorflow.keras.datasets import mnist
import time
# Set a random seed for reproducibility
np.random.seed(42)

In [4]:
!pip install trimesh
import trimesh

In [5]:
from sklearn.cluster import DBSCAN
import copy
import gc

In [6]:
from pathlib import Path
import pandas as pd
import os

import open3d as o3d
import sys
from dotenv import load_dotenv

import plotly.graph_objects as go
from plotly.subplots import make_subplots

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [7]:
!pip install --upgrade typing_extensions

  Using cached typing_extensions-4.13.2-py3-none-any.whl.metadata (3.0 kB)
Using cached typing_extensions-4.13.2-py3-none-any.whl (45 kB)
  Attempting uninstall: typing_extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.13.0 requires typing-extensions<4.6.0,>=3.6.6, but you have typing-extensions 4.13.2 which is incompatible.


In [8]:
pip install plotly ipywidgets

Note: you may need to restart the kernel to use updated packages.


The rest of the necessary packages are imported above.

Loading in the plant data for the individual maize and tomatos:

In [11]:
def load_in_plants(folder_name):
    root_dir = Path(folder_name)
    species = ["Maize", "Tomato"]
    indices = range(1, 8)  
    
    def load_plant(file_path):
        data = np.loadtxt(file_path)
        if data.shape[1] > 3:
            pts = data[:, :3]
            labels = data[:, 3:]
            return pts, labels
        else:
            return data, None
    
    all_plants = {}
    
    for s in species:
        for i in indices:
            folder_name = f"{s}0{i}"
            folder_path = root_dir / folder_name
            
            if not folder_path.exists():
                print(f"Folder missing: {folder_path}")
                continue
            
            all_plants[folder_name] = {}

            for file_path in sorted(folder_path.glob("*.txt")):
                pts, labels = load_plant(file_path)
                all_plants[folder_name][file_path.stem] = {
                    "points": pts,
                    "labels": labels
                }

    return all_plants


In [12]:
root_dir = Path("Pheno4d")   # change this to whatever your location is
species = ["Maize", "Tomato"]
indices = range(1, 8)  

def load_plant(file_path):
    data = np.loadtxt(file_path)
    if data.shape[1] > 3:
        pts = data[:, :3]
        labels = data[:, 3:]
        return pts, labels
    else:
        return data, None

all_plants = {}

for s in species:
    for i in indices:
        folder_name = f"{s}0{i}"
        folder_path = root_dir / folder_name
        
        if not folder_path.exists():
            print(f"Folder missing: {folder_path}")
            continue
        
        all_plants[folder_name] = {}
        print(f"🌿 Loading {folder_name}...")

        # loop to get all the evolutions of the certain plant 
        for file_path in sorted(folder_path.glob("*.txt")):
            pts, labels = load_plant(file_path)
            all_plants[folder_name][file_path.stem] = {
                "points": pts,
                "labels": labels
            }
            print(f"  ✅ Loaded {file_path.name} with {pts.shape[0]} points.")

print("✅ Done loading all plants!")


🌿 Loading Maize01...
  ✅ Loaded M01_0313_a.txt with 1554168 points.
  ✅ Loaded M01_0314.txt with 1641908 points.
  ✅ Loaded M01_0315_a.txt with 1301557 points.
  ✅ Loaded M01_0316.txt with 1026080 points.
  ✅ Loaded M01_0317_a.txt with 803846 points.
  ✅ Loaded M01_0318.txt with 337476 points.
  ✅ Loaded M01_0319_a.txt with 860826 points.
  ✅ Loaded M01_0320.txt with 527758 points.
  ✅ Loaded M01_0321_a.txt with 1114990 points.
  ✅ Loaded M01_0322.txt with 805391 points.
  ✅ Loaded M01_0324_a.txt with 1214597 points.
  ✅ Loaded M01_0325_a.txt with 1407956 points.
🌿 Loading Maize02...
  ✅ Loaded M02_0313_a.txt with 1729943 points.
  ✅ Loaded M02_0314.txt with 1432436 points.
  ✅ Loaded M02_0315_a.txt with 1441265 points.
  ✅ Loaded M02_0316.txt with 1407769 points.
  ✅ Loaded M02_0317_a.txt with 861631 points.
  ✅ Loaded M02_0318.txt with 496132 points.
  ✅ Loaded M02_0319_a.txt with 1133991 points.
  ✅ Loaded M02_0320.txt with 1117499 points.
  ✅ Loaded M02_0321_a.txt with 1444663 poin

This loads in all the plant data into a big dictionary all_plants that has the following example structure: all_plants["Maize01"]["M01_0313_a"]["points"] (or labels if any)

In [14]:
def voxel_downsample(points, voxel_size):
    
    cloud = o3d.geometry.PointCloud()
    cloud.points = o3d.utility.Vector3dVector(points)

    cloud = cloud.voxel_down_sample(voxel_size)
    
    return np.asarray(cloud.points)


In [15]:
def shift_up(plant):
    z_coordinate_min = pd.DataFrame(plant)[2].min()

    if z_coordinate_min >= 0:
        shift = np.repeat(-z_coordinate_min, plant.shape[0]).reshape(-1,1)
        padded = np.concatenate((np.zeros((plant.shape[0], 2)), shift), axis = 1)
        return plant + padded
    else:
        shift = np.repeat(abs(z_coordinate_min), plant.shape[0]).reshape(-1,1)
        padded = np.concatenate((np.zeros((plant.shape[0], 2)), shift), axis = 1)
        return plant + padded


In [16]:
def truncate(plant, value, labels=None):
    plant_df = pd.DataFrame(plant)
    labels_df = None

    indices = plant_df[plant_df[2] > value].index

    if labels is not None:
        labels_df = pd.DataFrame(labels).iloc[indices].to_numpy()

    truncated_points = plant_df.iloc[indices].to_numpy()

    del indices
    gc.collect()
    
    return [truncated_points, labels_df]
    

In [17]:
def clean_outliers(truncated_points, epsilon, cluster_size, plant_labels=None):
    DBSCAN_cluster = DBSCAN(eps=epsilon, min_samples=cluster_size).fit(truncated_points)

    labels = DBSCAN_cluster.labels_

    unique, counts = np.unique(labels[labels != -1], return_counts=True)
    largest_label = unique[np.argmax(counts)]

    indices = labels == largest_label

    cleaned_points = truncated_points[indices]
    
    final_plant_labels = None
    if plant_labels is not None:
        final_plant_labels = plant_labels[indices]

    del DBSCAN_cluster
    del labels
    del unique, counts
    del largest_label
    del indices
    gc.collect()

    return [cleaned_points, final_plant_labels]

In [18]:
def process_one_plant(plant):
    days = all_plants[plant]
    for point_cloud in days:
        downsampled = voxel_downsample(days[point_cloud]['points'], 3)
        shifted_up = shift_up(downsampled)
        truncated = truncate(shifted_up, 12)
        cleaned = clean_outliers(truncated[0], 10, 10)

        days[point_cloud]['points'] = cleaned[0]

    return days

    

In [19]:
def process_data(all_plants):
    for plant_evolution in all_plants:
        all_plants[plant_evolution] = process_one_plant(plant_evolution)

Runs the data processing procedure on all the plants and runs a timer.

In [21]:
import time
start = time.time()
processed_all_plants = process_data(all_plants)
end = time.time()
print(end-start)

348.6871144771576


Runs a slideshow of pop-up visualizations of all the processed plants.

In [83]:
import os
import numpy as np

def save_pointcloud_dict(data_dict, base_dir="."):
    """
    Creates folders for each top-level key and saves each points array
    in a txt file named after the second-level key.
    """
    for top_key, inner_dict in data_dict.items():
        folder_path = os.path.join(base_dir, top_key)
        os.makedirs(folder_path, exist_ok=True)

        for sub_key, content in inner_dict.items():
            points = content["points"]
            sub_key = sub_key.strip("_a")
            file_path = os.path.join(folder_path, f"{sub_key}.txt")

            # Save array as plaintext
            np.savetxt(file_path, points, fmt="%.8f")

            print(f"Saved: {file_path}")

save_pointcloud_dict(all_plants)


Saved: .\Maize01\M01_0313.txt
Saved: .\Maize01\M01_0314.txt
Saved: .\Maize01\M01_0315.txt
Saved: .\Maize01\M01_0316.txt
Saved: .\Maize01\M01_0317.txt
Saved: .\Maize01\M01_0318.txt
Saved: .\Maize01\M01_0319.txt
Saved: .\Maize01\M01_0320.txt
Saved: .\Maize01\M01_0321.txt
Saved: .\Maize01\M01_0322.txt
Saved: .\Maize01\M01_0324.txt
Saved: .\Maize01\M01_0325.txt
Saved: .\Maize02\M02_0313.txt
Saved: .\Maize02\M02_0314.txt
Saved: .\Maize02\M02_0315.txt
Saved: .\Maize02\M02_0316.txt
Saved: .\Maize02\M02_0317.txt
Saved: .\Maize02\M02_0318.txt
Saved: .\Maize02\M02_0319.txt
Saved: .\Maize02\M02_0320.txt
Saved: .\Maize02\M02_0321.txt
Saved: .\Maize02\M02_0322.txt
Saved: .\Maize02\M02_0324.txt
Saved: .\Maize02\M02_0325.txt
Saved: .\Maize03\M03_0313.txt
Saved: .\Maize03\M03_0314.txt
Saved: .\Maize03\M03_0315.txt
Saved: .\Maize03\M03_0316.txt
Saved: .\Maize03\M03_0317.txt
Saved: .\Maize03\M03_0319.txt
Saved: .\Maize03\M03_0320.txt
Saved: .\Maize03\M03_0321.txt
Saved: .\Maize03\M03_0322.txt
Saved: .\M

In [24]:
all_plants

{'Maize01': {'M01_0313_a': {'points': array([[ -23.14603295, -745.70623931,   56.12871265],
          [ -30.04851098, -740.69191341,   45.48586479],
          [ -23.27582846, -743.32831934,   46.81328405],
          [ -31.86564986, -740.2602044 ,   46.94545795],
          [ -33.98903406, -740.32983995,   52.31048371],
          [ -37.27685142, -730.27116575,   53.50262765],
          [ -37.86822222, -729.07046667,   66.80998403],
          [ -37.03508593, -730.361948  ,   32.20453278],
          [ -38.75125862, -731.51826322,   22.43943384],
          [ -37.45133323, -729.88198396,   47.29776871],
          [ -23.6637    , -743.5924    ,   48.76570625],
          [ -35.02528902, -730.29702905,   38.18646944],
          [ -37.01211615, -735.6998241 ,   41.19825516],
          [ -37.52557623, -729.57714093,   62.04201108],
          [ -31.90389231, -737.53972564,   39.91511394],
          [ -39.23212638, -732.43616819,   14.04145261],
          [ -38.94291489, -731.31160281,   16.8273697

In [77]:
keys = [key for key in all_plants][7:]

In [79]:
keys

['Tomato01',
 'Tomato02',
 'Tomato03',
 'Tomato04',
 'Tomato05',
 'Tomato06',
 'Tomato07']

In [81]:
for plant in all_plants["Tomato01"]:
    points = all_plants["Tomato01"][plant]['points']
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    o3d.visualization.draw_geometries([pcd]);

[Open3D WARNING] [ViewControl] SetViewPoint() failed because window height and width are not set.


KeyboardInterrupt: 

Here is a visualization to see the growth of a plant. We first store each plant evolution's point cloud days into a single array.

In [27]:
point_cloud_arrays = {}

for plant_evolution in all_plants:
    point_cloud_arrays[plant_evolution] = []
    for plant in all_plants[plant_evolution]:
        point_cloud_arrays[plant_evolution].append(all_plants[plant_evolution][plant]['points'])

In [28]:
import numpy as np
import plotly.graph_objects as go
from ipywidgets import interact, IntSlider

def visualize_plant_growth(plant_evolution):

    def show_cloud(idx):
        cloud = point_clouds[idx]
        x, y, z = cloud[:,0], cloud[:,1], cloud[:,2]
    
        fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z, mode='markers', marker=dict(size=2))])
    
        fig.update_layout(width=500, height=500,
            title=f"Plant Growth Stage {idx} — {len(cloud)} points",
            scene=dict(xaxis_title='X', yaxis_title='Y', zaxis_title='Z', aspectmode='data'))
    
        fig.show()
    
    point_clouds = point_cloud_arrays[plant_evolution]

    interact(show_cloud, idx=IntSlider(min=0, max=len(point_clouds)-1, step=1, value=0))


Here is an example of the visualization:

In [30]:
visualize_plant_growth('Maize01')

interactive(children=(IntSlider(value=0, description='idx', max=11), Output()), _dom_classes=('widget-interact…

In [31]:
import plotly.express as px

def stack_plants_visualization(plant_evolution):

    def colors_continuum(plant_evolution):
        num_stages = len(point_cloud_arrays[plant_evolution])
        colors = px.colors.qualitative.Plotly * ((num_stages // len(px.colors.qualitative.Plotly)) + 1)
        return num_stages, colors

    def show_growth(idx):
        fig = go.Figure()
    
        for i in range(idx + 1):
            cloud = point_clouds[i]
            fig.add_trace(go.Scatter3d(x=cloud[:,0], y=cloud[:,1], z=cloud[:,2], mode='markers',
                    marker=dict(size=3, color=colors[i], opacity=0.7), name=f"Stage {i}"))
        
        fig.update_layout(title=f"Plant Growth: stages 0 to {idx}",
            scene=dict(xaxis_title='X', yaxis_title='Y', zaxis_title='Z', aspectmode='data'), width=500, height=500)
        
        fig.show()
    
    point_clouds = point_cloud_arrays[plant_evolution]
    num_stages, colors = colors_continuum(plant_evolution)

    interact(show_growth, idx=IntSlider(min=0, max=num_stages-1, step=1, value=0))


In [32]:
stack_plants_visualization('Maize01')

interactive(children=(IntSlider(value=0, description='idx', max=11), Output()), _dom_classes=('widget-interact…

We now embed all the plants using LOTEmbedding. We do this via an embedding function using pyLOT.

In [34]:
from pyLOT.embed import LOTEmbedding

xr = np.random.normal(0, 1, size =(5000, 3)) # set reference measure with 5000 points

In [35]:
point_cloud_embeddings_LP = {}
#point_cloud_embeddings_sh = {}

for plant_evolution in point_cloud_arrays:
    point_cloud_embeddings_LP[plant_evolution] = LOTEmbedding.embed_point_clouds(xr, point_cloud_arrays[plant_evolution], sinkhorn = False)
    #point_cloud_embeddings_sh[plant_evolution] = LOTEmbedding.embed_point_clouds(xr, point_cloud_arrays[plant_evolution], xt_masses = None, sinkhorn = True, lambd = 0.05)
    

C:\Users\mycos\anaconda3\lib\site-packages\ot\lp\__init__.py:354: UserWarning:

numItermax reached before optimality. Try to increase numItermax.



same open3d visualization for the embedded point clouds.

In [37]:
# for plant_evolution in point_cloud_embeddings_LP:
#     for plant in point_cloud_embeddings_LP[plant_evolution]:
#         pcd = o3d.geometry.PointCloud()
#         pcd.points = o3d.utility.Vector3dVector(plant.reshape(5000,3))
#         o3d.visualization.draw_geometries([pcd]);